In [27]:
import pymupdf
import pymupdf4llm
import pandas as pd
import glob
import re
from numpy import float32
from datetime import datetime

In [28]:
all_pdf = glob.glob(r"PDF2\*.pdf")

In [29]:
pdf_document = r"PDF2\Как я стал core-разработчиком Python в 19 лет _ Хабр.pdf"
doc = pymupdf.open(pdf_document)
print("Исходный документ: ", doc.name)
print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
print(doc.metadata)
text = []
for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    page_text = page.get_text("text")
    print("Стр. ", current_page + 1, "\n")
    text.append(page_text)
    print(page_text)

Исходный документ:  PDF2\Как я стал core-разработчиком Python в 19 лет _ Хабр.pdf

Количество страниц: 4

------------------


{'format': 'PDF 1.4', 'title': 'Как я стал core-разработчиком Python в 19 лет / Хабр', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0', 'producer': 'Skia/PDF m135', 'creationDate': "D:20250429053726+00'00'", 'modDate': "D:20250429053726+00'00'", 'trapped': '', 'encryption': None}
Стр.  1 

20 сентября 2024 года я стал одним из участников команды разработки CPython.
Что значит CPython Core developer?
CPython Core Developer — это core-разработчик, имеющий официальные полномочия
вносить изменения в исходный код интерпретатора CPython, который является самой
распространённой реализацией языка Python. Коротко говоря, это те люди, которые могут
мержить пул-реквесты в репозитории CPython.
Мои первые шаги в CPython
Первым моим пул-рекв

In [30]:
text = ''.join(text).replace('\n',' ')
text

'20 сентября 2024 года я стал одним из участников команды разработки CPython. Что значит CPython Core developer? CPython Core Developer — это core-разработчик, имеющий официальные полномочия вносить изменения в исходный код интерпретатора CPython, который является самой распространённой реализацией языка Python. Коротко говоря, это те люди, которые могут мержить пул-реквесты в репозитории CPython. Мои первые шаги в CPython Первым моим пул-реквестом был фикс для документации. Это может показаться странным, но на самом деле такие изменения имеют смысл. Именно такие небольшие изменения позволяют познакомиться с workflow проекта: как правильно оформлять коммиты, как устроена система CI, как проходит процесс ревью, как добавлять записи в раздел What\'s New, и в целом — как построено взаимодействие с другими участниками разработки. Этот первый шаг особенно важен, потому что он снижает порог входа и помогает понять не только технические, но и социальные аспекты участия в проекте: как сообщест

In [31]:
def extract_markdown_text(document) -> str:
    return pymupdf4llm.to_markdown(document)
    

In [32]:
def parse_markdown(document: str):
    print(document)

In [33]:
def parse_article(document):
    doc = pymupdf.open(document)
    md = extract_markdown_text(document)
    lines = md.splitlines()
    
    # 1. Author
    author = None
    for line in lines:
        m = re.match(r"\*\*\[(.+?)\]\(", line)
        if m:
            author = m.group(1)
            break

    # 2. Company (if any)
    company = None
    for line in lines:
        if re.search(r"\d{2}\.\d{2}\.\d{4}", line) and "/" in line:
            company = line.split("/")[-1].strip()
            break

    # 3. Publish date (absolute or relative)
    publish_date = None
    for line in lines:
        m = re.match(r"(\d{2}\.\d{2}\.\d{4},\s*\d{2}:\d{2})", line)
        if m:
            publish_date = m.group(1); break
        m = re.match(r"(\d{1,2}\s+[А-Яа-я]{3}\s+в\s+\d{2}:\d{2})", line)
        if m:
            publish_date = m.group(1); break
        m = re.search(r"(\d+\s+(?:минут[аи]?|часов?|дней?)\s+назад)", line)
        if m:
            publish_date = doc.metadata['creationDate']; break
        
        if publish_date is None:
            publish_date = doc.metadata['creationDate']

    # 4. Description and Rating
    description = None
    rating = None
    for line in lines:
        parts = re.findall(r"\*\*(.+?)\*\*", line)
        # with company: expect 3+ bold parts: desc, time, rating
        if company and len(parts) >= 3 and re.match(r"^\d+K?$", parts[-1]):
            description = parts[0]
            raw = parts[-1]
            rating = int(raw[:-1])*1000 if raw.endswith("K") else int(raw)
            break
        # without company: expect 2 bold parts: time, rating
        if not company and len(parts) >= 2 and re.match(r"^\d+K?$", parts[-1]):
            raw = parts[-1]
            rating = int(raw[:-1])*1000 if raw.endswith("K") else int(raw)
            break

    # 5. Find the tags line (#### or #####) with comma-separated categories
    tags_idx = next(
        (i for i, line in enumerate(lines)
         if re.match(r"^#{4,5}\s*[^#].*\*,", line)),
        None
    )

    # 6. Capture article_text: all non-blank lines after tags_idx until the first '-----'
    article_text = ""
    if tags_idx is not None:
        content_lines = []
        for line in lines[tags_idx + 1:]:
            if line.strip().startswith("-----"):
                break
            if line.strip():
                content_lines.append(line.strip())
        article_text = "\n\n".join(content_lines)

    # 7. Build result dict
    result = {
        "author": author,
        "publish_date": publish_date,
        "rating": rating,
        "article_text": article_text
    }
    if company:
        result["company_name"] = company
        result["description"] = description

    return result

In [34]:
Brak = []
Author = []
PublishDate = []
Rating = []
ArticleText = []
CompanyName = []
CompanyDescription = []
for pdf in all_pdf:
    # try:  
        text = extract_markdown_text(pdf)
        print(parse_article(pdf))
        author, publish_date, rating, article_text, company_name, company_description = parse_article(pdf)
        Author.append(author)
        PublishDate.append(publish_date)
        Rating.append(rating)
        ArticleText.append(article_text)
        CompanyName.append(company_name)
        CompanyDescription.append(company_description)
    # except:
        # Brak.append(pdf)

{'author': 'TraPhro', 'publish_date': "D:20250318034441+00'00'", 'rating': None, 'article_text': '## От аутсайдера-задиры до лидера литографии\n\n#### В начале 1980-х годов небольшой городок Вельдховен тихо пожинал плоды своего соседа Эйндховена — процветающего промышленного центра благодаря присутствию голландского электронного гиганта Philips. Поскольку Philips продвигал технологические инновации в регионе, его влияние распространялось за пределы Эйндховена, способствуя волновому эффекту промышленной активности и технического опыта, которые достигли Вельдховена. С самого начала ASML был подразделением, которое многие в материнской компании считали скорее обузой, чем возможностью. Фактически, это был проблемный ребенок Philips — проект, который потратил более десятилетия на попытки проникнуть в фотолитографический бизнес без особого успеха. В Philips шутили, что создание ASML было по сути отложенным увольнением для переведенных в него сотрудников. Несмотря на внутренний скептицизм, Ph

ValueError: not enough values to unpack (expected 6, got 4)

In [ ]:
Brak

['PDF2\\ASML_ Создание самых сложных машин на Земле _ Хабр.pdf',
 'PDF2\\Cocoapods, Carthage, SPM как выбрать менеджер зависимостей в iOS.pdf',
 'PDF2\\Deutsche Telekom и Perplexity объявили о новом «AI Phone» стоимостью менее 1 000 долларов _ Хабр.pdf',
 'PDF2\\OSINT & Hacking — как работает фишинг для нельзяграма _ Хабр3.pdf',
 'PDF2\\Yandex N.V. запретили в\xa0течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр.pdf',
 'PDF2\\Быстрое начало работы с Gitlab CICD.pdf',
 'PDF2\\Газоразрядная лампа с символами, своими руками _ Хабр.pdf',
 'PDF2\\Геймеры и сообщества, повлиявшие на геймдев _ Хабр.pdf',
 'PDF2\\Генеральный директор Mozilla покинула свой пост _ Хабр.pdf',
 'PDF2\\Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр.pdf',
 'PDF2\\ИИ-агенты в Альфа-Банке_ нейросети создают автотесты без участия человека _ Хабр.pdf',
 'PDF2\\Инструменты наблюдаемости, о которых нужно знать в 2023 году.pdf',
 'PDF2\\История российской науки_ напишем вместе_ _ Хабр.pdf',
 'PDF2\\К

In [ ]:
from numpy import float32


# df = pd.DataFrame({
# 'Author': pd.Series(dtype=str),
# 'PublishDate': pd.Series(dtype=float32),
# 'DataPublish': pd.Series(dtype=str),
# 'Activity': pd.Series(dtype=str),
# 'TextArticle': pd.Series(dtype=str)
# })

df = pd.DataFrame({
    'Author': pd.Series(dtype=str),
    'PublishDate': pd.Series(dtype=str),
    'Rating': pd.Series(dtype=float32),
    'ArticleText': pd.Series(dtype=str),
    'CompanyName': pd.Series(dtype=str),
    'CompanyDescription': pd.Series(dtype=str)
})

In [ ]:
df['Author'] = Author
df['PublishDate'] = PublishDate
df['Rating'] = Rating
df['ArticleText'] = ArticleText
df['CompanyName'] = CompanyName
df['CompanyDescription'] = CompanyDescription

In [ ]:
df

,Author,PublishDate,Rating,ArticleText,CompanyName,CompanyDescription
0,author,publish_date,rating,article_text,company_name,description


In [ ]:
# pdf = extract_text_from_pdf(r"PDF2\ASML_ Создание самых сложных машин на Земле _ Хабр.pdf")

In [ ]:
# pars_pdf(pdf[0])